In [42]:
#Script completo para gerar Tabela Funcionario, gerar Tabela Gerente e atualizar ID_GERENTE para o seu IDFUNCIONARIO

import pandas as pd
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import random
import pyodbc
from datetime import date
from faker import Faker


server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"
conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

quant_funcionarios_gerados = 12534

try:
    with engine.connect() as conn:
            conn.execute(text("ALTER TABLE FUNCIONARIO DROP CONSTRAINT FK_FUNCIONARIO_GERENTE"))
            conn.commit()
            print(f"FK atualizada com sucesso!")
except Exception as e:
    print(f"Erro ao executar: {e}")

#Listas 
nomes = [
    "Maria", "Jose", "Ana", "Joao", "Antonio", "Francisco", "Carlos", "Paulo", "Pedro", "Lucas",
    "Luiz", "Marcos", "Luis", "Rafael", "Gabriel", "Daniel", "Marcelo", "Bruno", "Eduardo", "Felipe",
    "Raimundo", "Miguel", "Arthur", "Gustavo", "Leonardo", "Thiago", "Matheus", "Enzo", "Guilherme", "Davi",
    "Larissa", "Amanda", "Juliana", "Patricia", "Aline", "Camila", "Bruna", "Fernanda", "Isabela", "Mariana",
    "Tatiana", "Vanessa", "Carolina", "Bianca", "Clara", "Vitoria", "Leticia", "Laura", "Beatriz", "Julia",
    "Eliel", "Bernado", "Debora", "Shauanne", "Valdemira", "Rogeria", "Victor", "Paula", "Leandra", "Antonia",
    "Luiza", "Rafaela", "Gabriela", "Daniela", "Marcela", "Eduarda"
]

sobrenomes = [
    "Silva", "Santos", "Oliveira", "Souza", "Rodrigues", "Ferreira", "Alves", "Pereira", "Lima", "Gomes",
    "Costa", "Ribeiro", "Martins", "Carvalho", "Araujo", "Melo", "Barbosa", "Monteiro", "Cardoso", "Teixeira",
    "Mendes", "Nascimento", "Cavalcanti", "Dias", "Castro", "Campos", "Bezerra", "Fernandes", "Vieira", "Correia",
    "Sousa", "Machado", "Rocha", "Neves", "Nunes", "Moreira", "Medeiros", "Freitas", "Barros", "Pinto",
    "Andrade", "Moura", "Dantas", "Cunha", "Brito", "Leal", "Farias", "Peixoto", "Fonseca", "Azevedo"    
]

nomes_ruas = [
    "Rua Sao Joao", "Rua Santa Maria", "Rua Sao Paulo", "Rua Sao Pedro", "Rua Sao Jose",
    "Rua Sao Francisco", "Rua Santo Antonio", "Rua Sao Sebastiao", "Rua Santa Catarina",
    "Rua Luzia", "Rua Santa Rita", "Rua Santa Ana", "Rua Santa Terezinha", "Rua Santa Clara",
    "Rua Isabel", "Rua Santa Helena", "Rua Santa Cecilia", "Rua Santa Rosa", "Rua Santa Monica",
    "Rua Edwiges", "Rua Santa Ines", "Rua Santa Fe", "Rua Santa Lucia", "Rua Santa Efigenia",
    "Rua Barbara", "Rua Santa Angela", "Rua Santa Gertrudes", "Rua Santa Margarida",
    "Rua Marina", "Rua Santa Elisa", "Rua Santa Julia", "Rua Santa Zelia", "Rua Santa Zilda",
    "Rua Zulmira", "Rua Santa Zulene, Rua das Flores", "Rua da Paz", "Rua Sao Joao", "Rua Sao Paulo", "Rua Santo Antonio",
    "Rua 7 de Setembro", "Rua XV de Novembro", "Rua do Comercio", "Rua das Acacias", "Rua das Palmeiras",
    "Rua dos Cravos", "Rua do Sol", "Rua Lua Nova", "Rua da Esperanca", "Rua Sao Pedro",
    "Rua das Orquideas", "Rua das Rosas", "Rua Primavera", "Rua Santa Luzia", "Rua Sao Francisco",
    "Rua das Margaridas", "Rua do Bosque", "Rua das Aguas", "Rua Bela Vista", "Rua Santa Catarina",
    "Rua Dom Pedro I", "Rua Dom Pedro II", "Rua Marechal Deodoro", "Rua da Liberdade", "Rua do Lago",
    "Rua dos Girassois", "Rua Antonio Carlos", "Rua da Independencia", "Rua Presidente Vargas", "Rua das Laranjeiras",
    "Rua do Porto", "Rua Santos Dumont", "Rua Carlos Gomes", "Rua da Vitoria", "Rua do Carmo",
    "Rua do Campo", "Rua da Saudade", "Rua do Progresso", "Rua dos Bandeirantes", "Rua Coronel Antonio",
    "Rua Sao Jose", "Rua Benedito Calixto", "Rua dos Lirios", "Rua Sete Lagoas", "Rua Pedro Alvares Cabral",
    "Rua Castro Alves", "Rua Padre Anchieta", "Rua Machado de Assis", "Rua Monte Castelo", "Rua Humberto de Campos",
    "Rua dos Tucanos", "Rua das Violetas", "Rua do Ipe", "Rua dos Pinheiros", "Rua Jose Bonifacio",
    "Rua Getulio Vargas", "Rua Marechal Rondon", "Rua Visconde de Maua", "Rua do Horizonte", "Rua do Contorno",
    "Rua Rui Barbosa", "Rua Juscelino Kubitschek", "Rua Presidente Joao Goulart", "Rua General Osorio", "Rua dos Sonhos",
    "Rua das Figueiras", "Rua Goias", "Rua Alvorada", "Rua Bela Cintra", "Rua Piaui",
    "Rua Domingos Jorge Velho", "Rua General Carneiro", "Rua Dom Bosco", "Rua Miguel Couto", "Rua Alfredo Pujol",
    "Rua Frei Caneca", "Rua Dr. Arnaldo", "Rua Diogo Feijo", "Rua Regente Feijo", "Rua Conselheiro Lafaiete",
    "Rua Barao de Maua", "Rua Benjamin Constant", "Rua Santa Clara", "Rua Maranhao", "Rua Bahia",
    "Rua Amazonas", "Rua Acre", "Rua Sao Joaquim", "Rua dos Cedros", "Rua Tamoios",
    "Rua Jose de Alencar", "Rua Ceara", "Rua Camelia", "Rua das Cerejeiras", "Rua Japao"]

nomes_bairros = [
    "Centro", "Jardim", "Vila", "Sao Joao", "Sao Paulo", "Sao Pedro", "Sao Jose", "Sao Francisco",
    "Santo Antonio", "Sao Sebastiao", "Santa Maria", "Santa Catarina", "Santa Luzia", "Santa Rita",
    "Santa Ana", "Santa Terezinha", "Santa Clara", "Santa Isabel", "Santa Helena", "Santa Cecilia",
    "Santa Rosa", "Santa Monica", "Santa Edwiges", "Santa Ines", "Santa Fe", "Santa Lucia",
    "Santa Efigenia", "Santa Barbara", "Santa Angela", "Centro", "Jardim das Flores", "Vila Nova", "Jardim America", "Jardim Paulista",
    "Jardim Primavera", "Jardim Europa", "Jardim dos Ipes", "Jardim das Palmeiras", "Jardim Bela Vista",
    "Parque Industrial", "Jardim Alvorada", "Jardim Santa Clara", "Vila Esperanca", "Vila Maria",
    "Parque Sao Jorge", "Jardim Sao Francisco", "Jardim Santa Rita", "Vila Rica", "Vila Progresso",
    "Vila Matilde", "Vila Boa Vista", "Vila Sao Jose", "Parque dos Passaros", "Jardim Imperial",
    "Jardim Sao Luiz", "Jardim das Acacias", "Jardim Mariana", "Jardim do Lago", "Parque das Nacoes",
    "Parque dos Bandeirantes", "Jardim Atlantico", "Jardim Monte Castelo", "Jardim Eldorado", "Jardim Gloria",
    "Vila Oliveira", "Jardim Panorama", "Jardim Dom Bosco", "Vila Uniao", "Jardim Vitoria",
    "Jardim Novo Horizonte", "Vila Industrial", "Vila Augusta", "Jardim Botanico", "Jardim Sao Carlos",
    "Jardim Canada", "Jardim Santa Fe", "Jardim das Hortensias", "Parque Sao Bento", "Jardim dos Pinheiros",
    "Parque do Carmo", "Jardim Sao Joao", "Jardim Maria Helena", "Jardim Santa Rosa", "Jardim Felicidade",
    "Jardim Aurora", "Vila Santana", "Jardim Bela Vista", "Jardim das Magnolias", "Vila Medeiros",
    "Jardim dos Lirios", "Parque das Arvores", "Jardim Santa Terezinha", "Vila Nova Esperanca", "Jardim Santo Antonio",
    "Jardim Independencia", "Jardim Sao Jorge", "Jardim das Oliveiras", "Parque Estrela Dalva", "Jardim Mirante",
    "Vila Sao Pedro", "Parque Bela Vista", "Vila Nova Cachoeirinha", "Jardim Monte Alegre", "Parque Imperial",
    "Jardim Tropical", "Vila Santa Isabel", "Jardim Italia", "Jardim dos Cedros", "Jardim das Nacoes",
    "Vila Real", "Jardim dos Jacarandas", "Jardim Primavera", "Parque Paraiso", "Jardim Universo",
    "Jardim Santo Andre", "Jardim das Figueiras", "Vila Germanica", "Jardim Luz", "Vila Macedo",
    "Jardim Bonfim", "Jardim Colonial", "Jardim das Tulipas", "Vila Curuca", "Jardim Santa Maria"
]

cargos = [
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa',
'Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa',
'Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa',
'Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador',
'Gerente','Gerente','Gerente','Gerente','Gerente','Gerente','Gerente','Gerente','Gerente','Gerente',
'Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja',
'Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja',
'Analista de TI','Analista de TI','Analista de TI','Analista de TI','Analista de TI',
'Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH',
'Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH',
'Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro',
'Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo',
'Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo',
'Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque',
'Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque',
'Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque',
'Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente',
'Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente',
'Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente',
'Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança',
'Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança',
'Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro',
'Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro',
'Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral',
'Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral',
'Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral',
'Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza',
'Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza',
'Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas',
'Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas'
]

cargos1 = [
    'Repositor','Operador de caixa','Empacotador','Atendente','Ajudante geral','Auxiliar de limpeza']

cargos2 = [
    'Encarregado de estoque','Segurança','Padeiro','Açougueiro','Assistente Administrativo']

cargos3 = [
    'Supervisor de Loja','Promotor de vendas']

cargos4 = [
    'Analista de TI','Analista de RH','Analista financeiro','Gerente']

cidades_brasil = [ 
    "Sao Paulo", "Rio de Janeiro", "Brasilia", "Salvador", "Fortaleza",
    "Belo Horizonte", "Manaus", "Curitiba", "Recife", "Goiania",
    "Porto Alegre", "Guarulhos", "Campinas", "Sao Luis",
    "Sao Goncalo", "Maceio", "Duque de Caxias", "Natal", "Teresina",
    "Sao Bernardo do Campo", "Joao Pessoa", "Osasco",
    "Jaboatao dos Guararapes", "Ribeirao Preto", "Uberlandia", "Contagem", "Sorocaba",
    "Aracaju", "Feira de Santana", "Cuiaba", "Joinville", "Aparecida de Goiania",
    "Londrina", "Juiz de Fora", "Ananindeua", "Belford Roxo", "Niteroi",
    "Sao Joao de Meriti", "Campos dos Goytacazes", "Serra", "Caxias do Sul", "Maua",
    "Macapa", "Piracicaba", "Canoas", "Bauru", "Sao Jose dos Campos"]

df_maiores_cidades = pd.read_sql("SELECT IDCIDADE FROM CIDADE WHERE NOME IN ({})".format(','.join(f"'{cidade}'" for cidade in cidades_brasil)), engine)['IDCIDADE'].tolist()

# Função para gerar um nome completo, gênero e e-mail
def gerar_nome_completo():
    nome = random.choice(nomes)
    sobrenome = random.choice(sobrenomes)
    sobrenome2 = random.choice(sobrenomes)
    sobrenome3 = random.choice(sobrenomes)
    nomecompleto = f'{nome} {sobrenome} {sobrenome2} {sobrenome3}'
    cargo = random.choice(cargos)
    if cargo in cargos1:
        salario = round(random.uniform(1550,1975),2)
    elif cargo in cargos2:
        salario = round(random.uniform(1815,2380),2)
    elif cargo in cargos3:
        salario = round(random.uniform(2472,3257),2)
    else:
        salario = round(random.uniform(3441,4115),2)
    rua = random.choice(nomes_ruas)
    num = random.randint(5, 5000)
    bairro = random.choice(nomes_bairros)
    endereco = f"{rua} - {num} - {bairro}"
    id_cidade = random.choice(df_maiores_cidades)
    total_lojas_db = (engine.connect().execute(text("SELECT COUNT(IDLOJA) FROM [MERCADO_OLTP].[dbo].LOJA"))).scalar()
    id_loja = random.randint(1, total_lojas_db)
    id_gerente = 1

    if cargo in ['Repositor','Operador de caixa','Empacotador','Encarregado de estoque','Segurança','Açougueiro','Padeiro','Ajudante geral','Auxiliar de limpeza']:
        id_departamento = 4
    elif cargo in ['Supervisor de Loja','Assistente Administrativo','Gerente']:
        id_departamento = 1
    elif cargo in ['Atendente','Promotor de vendas']:
        id_departamento = 2
    elif cargo in ['Analista financeiro']:
        id_departamento = 3
    elif cargo in ['Analista de RH']:
        id_departamento = 5
    else:
        id_departamento = 6
        
    variavelcontratacao = random.randint(1,1000)
    fake = Faker() #variavel para usar Faker
    if variavelcontratacao <=865:
        situacao = 'Ativa'
        data_admissao = fake.date_time_between(start_date='-15y',end_date='now')
        data_demissao = None
        data_criacao = data_admissao
        data_atlz = None
        versao = 1
    else:
        situacao = 'Inativa'
        data_demissao = fake.date_time_between(start_date='-14y',end_date='now')
        data_admissao = fake.date_time_between(start_date='-15y',end_date=data_demissao)
        data_criacao = data_admissao
        data_atlz = data_demissao
        versao = 2
    
    return nomecompleto, cargo, endereco, salario, id_cidade, id_loja, id_gerente, id_departamento,situacao, data_admissao, data_demissao, data_criacao, data_atlz, versao

# Gerar 1000 nomes completos com gênero e e-mail
dados_nomes = [gerar_nome_completo() for _ in range(quant_funcionarios_gerados)]
df_funcionarios = pd.DataFrame(dados_nomes, columns=['NOME', 'CARGO', 'ENDERECO','SALARIO', 'ID_CIDADE', 'ID_LOJA', 'ID_GERENTE', 'ID_DEPARTAMENTO','SITUACAO', 'DATA_ADMISSAO','DATA_DEMISSAO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO'])

#Conta a quantidade de gerente gerados, e atualiza a coluna ID_GERENTE de todos os funcionarios
#print('df_funcionaarios completo')
#display(df_funcionarios)
#quantidade_gerentes = int(df_funcionarios['CARGO'].value_counts()['Gerente'])
#print(f'quantidade de gerente:{quantidade_gerentes}')
#filtro = (df_funcionarios['CARGO'] == 'Gerente') & (df_funcionarios['SITUACAO'] == 'Ativa')
quantidade_gerentes = len(df_funcionarios[(df_funcionarios['CARGO'] == 'Gerente') & (df_funcionarios['SITUACAO'] == 'Ativa')])
#print(f'quantidade de gerente na ativa:{}')
lista_id_gerente = [random.randint(1, quantidade_gerentes) for _ in range(len(df_funcionarios))]
#print('lista de id gerentes')
#display(lista_id_gerente)
df_funcionarios['ID_GERENTE'] = lista_id_gerente
#display(df_funcionarios)

#Função para gerar números aleatórios únicos
def gerar_numeros_unicos(intervalo_inicio, intervalo_fim, quantidade):
    if quantidade > (intervalo_fim - intervalo_inicio + 1):
        raise ValueError("A quantidade de números únicos solicitada excede o intervalo disponível.")
    numeros_unicos = random.sample(range(intervalo_inicio, intervalo_fim + 1), quantidade)
    return numeros_unicos
#Valores de entrada
intervalo_inicio = 10000000000  # Início do intervalo
intervalo_fim =    99999999999  # Fim do intervalo
quantidade_registros = quant_funcionarios_gerados  # Quantidade de registros no DataFrame

lista_cpf = gerar_numeros_unicos(intervalo_inicio, intervalo_fim, quantidade_registros)

#Filtra os valores de CPF gerados para funcionarios, garantind oque nao tenha iguais dos cpfs de clientes
df_funcionarios["CPF"] = pd.Series(lista_cpf).astype(str).tolist()
cpf_clientes = pd.read_sql("SELECT [CPF] FROM [dbo].[CLIENTE]", engine.connect())["CPF"].astype(str).tolist()
df_funcionarios = df_funcionarios[~df_funcionarios["CPF"].isin(cpf_clientes)]

#Remove espacos desnecessarios nos dados
df_funcionarios = df_funcionarios.apply(lambda col: col.str.replace(r'\s+', ' ', regex=True).str.strip() if col.dtype == 'object' and col.apply(lambda x: isinstance(x, str)).any() else col)

df_funcionario = pd.DataFrame({

                'NOME':              df_funcionarios['NOME'],
                'CPF':               df_funcionarios['CPF'],
                'CARGO':             df_funcionarios['CARGO'],
                'SALARIO':           df_funcionarios['SALARIO'],
                'ENDERECO':          df_funcionarios['ENDERECO'],
                'ID_CIDADE':         df_funcionarios['ID_CIDADE'],
                'ID_LOJA':           df_funcionarios['ID_LOJA'],
                'ID_GERENTE':        df_funcionarios['ID_GERENTE'],
                'ID_DEPARTAMENTO':   df_funcionarios['ID_DEPARTAMENTO'],
                'SITUACAO':          df_funcionarios['SITUACAO'],
                'DATA_ADMISSAO':     df_funcionarios['DATA_ADMISSAO'].dt.date,
                'DATA_DEMISSAO':     df_funcionarios['DATA_DEMISSAO'].dt.date,
                'DATA_CRIACAO':      df_funcionarios['DATA_CRIACAO'].dt.date,
                'DATA_ATLZ':         df_funcionarios['DATA_ATLZ'].dt.date,
                'VERSAO':            df_funcionarios['VERSAO'],
})

tipo_dados_funcionarios = {
                'NOME':              VARCHAR(100),
                'CPF':               VARCHAR(11),
                'CARGO':             VARCHAR(50),
                'SALARIO':           NUMERIC(18,2),
                'ENDERECO':          VARCHAR(100),
                'ID_CIDADE':         INTEGER,
                'ID_LOJA':           INTEGER,
                'ID_GERENTE':        INTEGER,
                'ID_DEPARTAAMENTO':  INTEGER,
                'SITUACAO':          VARCHAR(10),
                'DATA_ADMISSAO':     DATE,
                'DATA_DEMISSAO':     DATE,
                'DATA_CRIACAO':      DATE,
                'DATA_ATLZ':         DATE,
                'VERSAO':            INTEGER,
}

with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE FUNCIONARIO"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()
#
df_funcionario.to_sql(name='FUNCIONARIO',con=engine,if_exists='append',index=False,dtype=tipo_dados_funcionarios)

#Script para gerar TABELA GERENTES A PARTIR DA FUNCIONARIOS 

df_gerentes = pd.read_sql(text(f"SELECT * FROM FUNCIONARIO WHERE CARGO = 'Gerente' and SITUACAO = 'Ativa'"),engine)

df_gerente = pd.DataFrame({

                'NOME':              df_gerentes['NOME'],
                'ID_LOJA':           df_gerentes['ID_LOJA'],
                'ID_DEPARTAMENTO':   df_gerentes['ID_DEPARTAMENTO'],
                'DATA_CRIACAO':      df_gerentes['DATA_CRIACAO'],
                'DATA_ATLZ':         None,
                'VERSAO':            1,
})

tipo_dados_gerente = {
                'NOME':              VARCHAR(100),
                'ID_LOJA':           INTEGER,
                'ID_DEPARTAAMENTO':  INTEGER,
                'DATA_CRIACAO':      DATE,
                'DATA_ATLZ':         DATE,
                'VERSAO':            INTEGER,
}

with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE GERENTE"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()

df_gerente.to_sql(name='GERENTE',con=engine,if_exists='append',index=False,dtype=tipo_dados_gerente)

#Script para atualizar o IDGERENTE do gerente na tabela FUNCIONARIO

df_gerentes   = pd.read_sql(text(f"SELECT * FROM FUNCIONARIO WHERE CARGO = 'Gerente' and SITUACAO = 'Ativa'"),engine)
df_gerentes2  = pd.read_sql(text(f"SELECT IDGERENTE, NOME FROM GERENTE"),engine)

conn = engine
df = df_gerentes
df2 = df_gerentes2
update_table = 'FUNCIONARIO'
chave = 'IDFUNCIONARIO'
atualizacao = 'ID_GERENTE'

def update_table_from_dataframe(engine, df, df2, update_table, chave, atualizacao):
    try:
        with engine.connect() as conn:
            for index, row in df.iterrows():
                valor_procurado = row['NOME']
                
                id_gerente_final = int(df2.loc[df2['NOME'] == valor_procurado, 'IDGERENTE'].values[0])
                # Monta o comando UPDATE
                update_query = text(f"""
                    UPDATE {update_table}
                    SET {atualizacao} = :id_gerente_final
                    WHERE {chave} = :id_funcionario
                """)
                # Executa o UPDATE
                conn.execute(update_query, {'id_gerente_final': id_gerente_final, 'id_funcionario': row[chave]})
            conn.commit()
            print(f"Tabela '{update_table}' atualizada com sucesso!")
    except Exception as e:
        print(f"Erro ao executar UPDATE: {e}")

update_table_from_dataframe(conn, df_gerentes, df_gerentes2, update_table, chave, atualizacao)

try:
    with engine.connect() as conn:
            conn.execute(text("ALTER TABLE FUNCIONARIO ADD CONSTRAINT FK_FUNCIONARIO_GERENTE FOREIGN KEY(ID_GERENTE) REFERENCES GERENTE(IDGERENTE)"))
            conn.commit()
            print(f"FK adicionada com sucesso!")
except Exception as e:
    print(f"Erro ao executar: {e}")


Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida
FK atualizada com sucesso!
Tabela 'FUNCIONARIO' atualizada com sucesso!
FK adicionada com sucesso!


In [ ]:
#Script para gerar Tabela Funcionario

import pandas as pd
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import random
import pyodbc
from datetime import date
from faker import Faker

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"
#user = ""
#senha = ""

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
#conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

quant_funcionarios_gerados = 1000

#Listas 
nomes = [
    "Maria", "Jose", "Ana", "Joao", "Antonio", "Francisco", "Carlos", "Paulo", "Pedro", "Lucas",
    "Luiz", "Marcos", "Luis", "Rafael", "Gabriel", "Daniel", "Marcelo", "Bruno", "Eduardo", "Felipe",
    "Raimundo", "Miguel", "Arthur", "Gustavo", "Leonardo", "Thiago", "Matheus", "Enzo", "Guilherme", "Davi",
    "Larissa", "Amanda", "Juliana", "Patricia", "Aline", "Camila", "Bruna", "Fernanda", "Isabela", "Mariana",
    "Tatiana", "Vanessa", "Carolina", "Bianca", "Clara", "Vitoria", "Leticia", "Laura", "Beatriz", "Julia",
    "Eliel", "Bernado", "Debora", "Shauanne", "Valdemira", "Rogeria", "Victor", "Paula", "Leandra", "Antonia",
    "Luiza", "Rafaela", "Gabriela", "Daniela", "Marcela", "Eduarda"
]

sobrenomes = [
    "Silva", "Santos", "Oliveira", "Souza", "Rodrigues", "Ferreira", "Alves", "Pereira", "Lima", "Gomes",
    "Costa", "Ribeiro", "Martins", "Carvalho", "Araujo", "Melo", "Barbosa", "Monteiro", "Cardoso", "Teixeira",
    "Mendes", "Nascimento", "Cavalcanti", "Dias", "Castro", "Campos", "Bezerra", "Fernandes", "Vieira", "Correia",
    "Sousa", "Machado", "Rocha", "Neves", "Nunes", "Moreira", "Medeiros", "Freitas", "Barros", "Pinto",
    "Andrade", "Moura", "Dantas", "Cunha", "Brito", "Leal", "Farias", "Peixoto", "Fonseca", "Azevedo"    
]

nomes_ruas = [
    "Rua Sao Joao", "Rua Santa Maria", "Rua Sao Paulo", "Rua Sao Pedro", "Rua Sao Jose",
    "Rua Sao Francisco", "Rua Santo Antonio", "Rua Sao Sebastiao", "Rua Santa Catarina",
    "Rua Luzia", "Rua Santa Rita", "Rua Santa Ana", "Rua Santa Terezinha", "Rua Santa Clara",
    "Rua Isabel", "Rua Santa Helena", "Rua Santa Cecilia", "Rua Santa Rosa", "Rua Santa Monica",
    "Rua Edwiges", "Rua Santa Ines", "Rua Santa Fe", "Rua Santa Lucia", "Rua Santa Efigenia",
    "Rua Barbara", "Rua Santa Angela", "Rua Santa Gertrudes", "Rua Santa Margarida",
    "Rua Marina", "Rua Santa Elisa", "Rua Santa Julia", "Rua Santa Zelia", "Rua Santa Zilda",
    "Rua Zulmira", "Rua Santa Zulene, Rua das Flores", "Rua da Paz", "Rua Sao Joao", "Rua Sao Paulo", "Rua Santo Antonio",
    "Rua 7 de Setembro", "Rua XV de Novembro", "Rua do Comercio", "Rua das Acacias", "Rua das Palmeiras",
    "Rua dos Cravos", "Rua do Sol", "Rua Lua Nova", "Rua da Esperanca", "Rua Sao Pedro",
    "Rua das Orquideas", "Rua das Rosas", "Rua Primavera", "Rua Santa Luzia", "Rua Sao Francisco",
    "Rua das Margaridas", "Rua do Bosque", "Rua das Aguas", "Rua Bela Vista", "Rua Santa Catarina",
    "Rua Dom Pedro I", "Rua Dom Pedro II", "Rua Marechal Deodoro", "Rua da Liberdade", "Rua do Lago",
    "Rua dos Girassois", "Rua Antonio Carlos", "Rua da Independencia", "Rua Presidente Vargas", "Rua das Laranjeiras",
    "Rua do Porto", "Rua Santos Dumont", "Rua Carlos Gomes", "Rua da Vitoria", "Rua do Carmo",
    "Rua do Campo", "Rua da Saudade", "Rua do Progresso", "Rua dos Bandeirantes", "Rua Coronel Antonio",
    "Rua Sao Jose", "Rua Benedito Calixto", "Rua dos Lirios", "Rua Sete Lagoas", "Rua Pedro Alvares Cabral",
    "Rua Castro Alves", "Rua Padre Anchieta", "Rua Machado de Assis", "Rua Monte Castelo", "Rua Humberto de Campos",
    "Rua dos Tucanos", "Rua das Violetas", "Rua do Ipe", "Rua dos Pinheiros", "Rua Jose Bonifacio",
    "Rua Getulio Vargas", "Rua Marechal Rondon", "Rua Visconde de Maua", "Rua do Horizonte", "Rua do Contorno",
    "Rua Rui Barbosa", "Rua Juscelino Kubitschek", "Rua Presidente Joao Goulart", "Rua General Osorio", "Rua dos Sonhos",
    "Rua das Figueiras", "Rua Goias", "Rua Alvorada", "Rua Bela Cintra", "Rua Piaui",
    "Rua Domingos Jorge Velho", "Rua General Carneiro", "Rua Dom Bosco", "Rua Miguel Couto", "Rua Alfredo Pujol",
    "Rua Frei Caneca", "Rua Dr. Arnaldo", "Rua Diogo Feijo", "Rua Regente Feijo", "Rua Conselheiro Lafaiete",
    "Rua Barao de Maua", "Rua Benjamin Constant", "Rua Santa Clara", "Rua Maranhao", "Rua Bahia",
    "Rua Amazonas", "Rua Acre", "Rua Sao Joaquim", "Rua dos Cedros", "Rua Tamoios",
    "Rua Jose de Alencar", "Rua Ceara", "Rua Camelia", "Rua das Cerejeiras", "Rua Japao"]

nomes_bairros = [
    "Centro", "Jardim", "Vila", "Sao Joao", "Sao Paulo", "Sao Pedro", "Sao Jose", "Sao Francisco",
    "Santo Antonio", "Sao Sebastiao", "Santa Maria", "Santa Catarina", "Santa Luzia", "Santa Rita",
    "Santa Ana", "Santa Terezinha", "Santa Clara", "Santa Isabel", "Santa Helena", "Santa Cecilia",
    "Santa Rosa", "Santa Monica", "Santa Edwiges", "Santa Ines", "Santa Fe", "Santa Lucia",
    "Santa Efigenia", "Santa Barbara", "Santa Angela", "Centro", "Jardim das Flores", "Vila Nova", "Jardim America", "Jardim Paulista",
    "Jardim Primavera", "Jardim Europa", "Jardim dos Ipes", "Jardim das Palmeiras", "Jardim Bela Vista",
    "Parque Industrial", "Jardim Alvorada", "Jardim Santa Clara", "Vila Esperanca", "Vila Maria",
    "Parque Sao Jorge", "Jardim Sao Francisco", "Jardim Santa Rita", "Vila Rica", "Vila Progresso",
    "Vila Matilde", "Vila Boa Vista", "Vila Sao Jose", "Parque dos Passaros", "Jardim Imperial",
    "Jardim Sao Luiz", "Jardim das Acacias", "Jardim Mariana", "Jardim do Lago", "Parque das Nacoes",
    "Parque dos Bandeirantes", "Jardim Atlantico", "Jardim Monte Castelo", "Jardim Eldorado", "Jardim Gloria",
    "Vila Oliveira", "Jardim Panorama", "Jardim Dom Bosco", "Vila Uniao", "Jardim Vitoria",
    "Jardim Novo Horizonte", "Vila Industrial", "Vila Augusta", "Jardim Botanico", "Jardim Sao Carlos",
    "Jardim Canada", "Jardim Santa Fe", "Jardim das Hortensias", "Parque Sao Bento", "Jardim dos Pinheiros",
    "Parque do Carmo", "Jardim Sao Joao", "Jardim Maria Helena", "Jardim Santa Rosa", "Jardim Felicidade",
    "Jardim Aurora", "Vila Santana", "Jardim Bela Vista", "Jardim das Magnolias", "Vila Medeiros",
    "Jardim dos Lirios", "Parque das Arvores", "Jardim Santa Terezinha", "Vila Nova Esperanca", "Jardim Santo Antonio",
    "Jardim Independencia", "Jardim Sao Jorge", "Jardim das Oliveiras", "Parque Estrela Dalva", "Jardim Mirante",
    "Vila Sao Pedro", "Parque Bela Vista", "Vila Nova Cachoeirinha", "Jardim Monte Alegre", "Parque Imperial",
    "Jardim Tropical", "Vila Santa Isabel", "Jardim Italia", "Jardim dos Cedros", "Jardim das Nacoes",
    "Vila Real", "Jardim dos Jacarandas", "Jardim Primavera", "Parque Paraiso", "Jardim Universo",
    "Jardim Santo Andre", "Jardim das Figueiras", "Vila Germanica", "Jardim Luz", "Vila Macedo",
    "Jardim Bonfim", "Jardim Colonial", "Jardim das Tulipas", "Vila Curuca", "Jardim Santa Maria"
]

cargos = [
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa',
'Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa',
'Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa',
'Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador',
'Gerente','Gerente','Gerente','Gerente','Gerente','Gerente','Gerente','Gerente','Gerente','Gerente',
'Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja',
'Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja',
'Analista de TI','Analista de TI','Analista de TI','Analista de TI','Analista de TI',
'Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH',
'Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH',
'Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro',
'Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo',
'Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo',
'Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque',
'Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque',
'Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque',
'Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente',
'Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente',
'Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente',
'Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança',
'Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança',
'Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro',
'Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro',
'Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral',
'Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral',
'Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral',
'Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza',
'Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza',
'Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas',
'Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas'
]

cargos1 = [
    'Repositor','Operador de caixa','Empacotador','Atendente','Ajudante geral','Auxiliar de limpeza']

cargos2 = [
    'Encarregado de estoque','Segurança','Padeiro','Açougueiro','Assistente Administrativo']

cargos3 = [
    'Supervisor de Loja','Promotor de vendas']

cargos4 = [
    'Analista de TI','Analista de RH','Analista financeiro','Gerente']

cidades_brasil = [ 
    "Sao Paulo", "Rio de Janeiro", "Brasilia", "Salvador", "Fortaleza",
    "Belo Horizonte", "Manaus", "Curitiba", "Recife", "Goiania",
    "Porto Alegre", "Guarulhos", "Campinas", "Sao Luis",
    "Sao Goncalo", "Maceio", "Duque de Caxias", "Natal", "Teresina",
    "Sao Bernardo do Campo", "Joao Pessoa", "Osasco",
    "Jaboatao dos Guararapes", "Ribeirao Preto", "Uberlandia", "Contagem", "Sorocaba",
    "Aracaju", "Feira de Santana", "Cuiaba", "Joinville", "Aparecida de Goiania",
    "Londrina", "Juiz de Fora", "Ananindeua", "Belford Roxo", "Niteroi",
    "Sao Joao de Meriti", "Campos dos Goytacazes", "Serra", "Caxias do Sul", "Maua",
    "Macapa", "Piracicaba", "Canoas", "Bauru", "Sao Jose dos Campos"]

df_maiores_cidades = pd.read_sql("SELECT IDCIDADE FROM CIDADE WHERE NOME IN ({})".format(','.join(f"'{cidade}'" for cidade in cidades_brasil)), engine)['IDCIDADE'].tolist()

# Função para gerar um nome completo, gênero e e-mail
def gerar_nome_completo():
    nome = random.choice(nomes)
    sobrenome = random.choice(sobrenomes)
    sobrenome2 = random.choice(sobrenomes)
    sobrenome3 = random.choice(sobrenomes)
    nomecompleto = f'{nome} {sobrenome} {sobrenome2} {sobrenome3}'
    cargo = random.choice(cargos)
    if cargo in cargos1:
        salario = round(random.uniform(1550,1975),2)
    elif cargo in cargos2:
        salario = round(random.uniform(1815,2380),2)
    elif cargo in cargos3:
        salario = round(random.uniform(2472,3257),2)
    else:
        salario = round(random.uniform(3441,4115),2)
    rua = random.choice(nomes_ruas)
    num = random.randint(5, 5000)
    bairro = random.choice(nomes_bairros)
    endereco = f"{rua} - {num} - {bairro}"
    id_cidade = random.choice(df_maiores_cidades)
    total_lojas_db = (engine.connect().execute(text("SELECT COUNT(IDLOJA) FROM [MERCADO_OLTP].[dbo].LOJA"))).scalar()
    id_loja = random.randint(1, total_lojas_db)
    
    if cargo == 'Gerente':
        id_gerente = None
    else:
        id_gerente = random.randint(1,280)

    if cargo in ['Repositor','Operador de caixa','Empacotador','Encarregado de estoque','Segurança','Açougueiro','Padeiro','Ajudante geral','Auxiliar de limpeza']:
        id_departamento = 4
    elif cargo in ['Supervisor de Loja','Assistente Administrativo','Gerente']:
        id_departamento = 1
    elif cargo in ['Atendente','Promotor de vendas']:
        id_departamento = 2
    elif cargo in ['Analista financeiro']:
        id_departamento = 3
    elif cargo in ['Analista de RH']:
        id_departamento = 5
    else:
        id_departamento = 6
        
    variavelcontratacao = random.randint(1,1000)
    fake = Faker() #variavel para usar Faker
    if variavelcontratacao <=865:
        situacao = 'Ativa'
        data_admissao = fake.date_time_between(start_date='-15y',end_date='now')
        data_demissao = None
        data_criacao = data_admissao
        data_atlz = None
        versao = 1
    else:
        situacao = 'Inativa'
        data_demissao = fake.date_time_between(start_date='-14y',end_date='now')
        data_admissao = fake.date_time_between(start_date='-15y',end_date=data_demissao)
        data_criacao = data_admissao
        data_atlz = data_demissao
        versao = 2
    
    return nomecompleto, cargo, endereco, salario, id_cidade, id_loja, id_gerente, id_departamento,situacao, data_admissao, data_demissao, data_criacao, data_atlz, versao

# Gerar 1000 nomes completos com gênero e e-mail
dados_nomes = [gerar_nome_completo() for _ in range(quant_funcionarios_gerados)]
df_funcionarios = pd.DataFrame(dados_nomes, columns=['NOME', 'CARGO', 'ENDERECO','SALARIO', 'ID_CIDADE', 'ID_LOJA', 'ID_GERENTE', 'ID_DEPARTAMENTO','SITUACAO', 'DATA_ADMISSAO','DATA_DEMISSAO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO'])

#Função para gerar números aleatórios únicos
def gerar_numeros_unicos(intervalo_inicio, intervalo_fim, quantidade):
    if quantidade > (intervalo_fim - intervalo_inicio + 1):
        raise ValueError("A quantidade de números únicos solicitada excede o intervalo disponível.")
    numeros_unicos = random.sample(range(intervalo_inicio, intervalo_fim + 1), quantidade)
    return numeros_unicos
#Valores de entrada
intervalo_inicio = 10000000000  # Início do intervalo
intervalo_fim =    99999999999  # Fim do intervalo
quantidade_registros = quant_funcionarios_gerados  # Quantidade de registros no DataFrame

lista_cpf = gerar_numeros_unicos(intervalo_inicio, intervalo_fim, quantidade_registros)

df_funcionarios["CPF"] = lista_cpf

df_funcionarios = df_funcionarios.apply(lambda col: col.str.replace(r'\s+', ' ', regex=True).str.strip() if col.dtype == 'object' and col.apply(lambda x: isinstance(x, str)).any() else col)

df_funcionario = pd.DataFrame({

                'NOME':              df_funcionarios['NOME'],
                'CPF':               df_funcionarios['CPF'],
                'CARGO':             df_funcionarios['CARGO'],
                'SALARIO':           df_funcionarios['SALARIO'],
                'ENDERECO':          df_funcionarios['ENDERECO'],
                'ID_CIDADE':         df_funcionarios['ID_CIDADE'],
                'ID_LOJA':           df_funcionarios['ID_LOJA'],
                'ID_GERENTE':        df_funcionarios['ID_GERENTE'],
                'ID_DEPARTAMENTO':   df_funcionarios['ID_DEPARTAMENTO'],
                'SITUACAO':          df_funcionarios['SITUACAO'],
                'DATA_ADMISSAO':     df_funcionarios['DATA_ADMISSAO'].dt.date,
                'DATA_DEMISSAO':     df_funcionarios['DATA_DEMISSAO'].dt.date,
                'DATA_CRIACAO':      df_funcionarios['DATA_CRIACAO'].dt.date,
                'DATA_ATLZ':         df_funcionarios['DATA_ATLZ'].dt.date,
                'VERSAO':            df_funcionarios['VERSAO'],
})

display(df_funcionario)

tipo_dados_funcionarios = {
                'NOME':              VARCHAR(100),
                'CPF':               VARCHAR(11),
                'CARGO':             VARCHAR(50),
                'SALARIO':           NUMERIC(18,2),
                'ENDERECO':          VARCHAR(100),
                'ID_CIDADE':         INTEGER,
                'ID_LOJA':           INTEGER,
                'ID_GERENTE':        INTEGER,
                'ID_DEPARTAAMENTO':  INTEGER,
                'SITUACAO':          VARCHAR(10),
                'DATA_ADMISSAO':     DATE,
                'DATA_DEMISSAO':     DATE,
                'DATA_CRIACAO':      DATE,
                'DATA_ATLZ':         DATE,
                'VERSAO':            INTEGER,
}

with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE FUNCIONARIO"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()

df_funcionario.to_sql(name='FUNCIONARIO',con=engine,if_exists='append',index=False,dtype=tipo_dados_funcionarios)

engine.dispose()


In [ ]:
#Script para atualizar o IDGERENTE do gerente na tabela FUNCIONARIO

import pyodbc
import pandas as pd
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

df_gerentes = pd.read_sql(text(f"SELECT * FROM FUNCIONARIO WHERE CARGO = 'Gerente' and SITUACAO = 'Ativa'"),engine)
display(df_gerentes)

conn = engine
df = df_gerentes
update_table = 'FUNCIONARIO'
chave = 'IDFUNCIONARIO'
atualizacao = 'ID_GERENTE'

def update_table_from_dataframe(engine, df, update_table, chave, atualizacao):
    try:
        with engine.connect() as conn:
            for index, row in df.iterrows():
                # Monta o comando UPDATE
                update_query = text(f"""
                    UPDATE {update_table}
                    SET {atualizacao} = :id_gerente
                    WHERE {chave} = :id_funcionario
                """)
                # Executa o UPDATE
                conn.execute(update_query, {'id_gerente': row['IDFUNCIONARIO'], 'id_funcionario': row['IDFUNCIONARIO']})
            conn.commit()
            print(f"Tabela '{update_table}' atualizada com sucesso!")
    except Exception as e:
        print(f"Erro ao executar UPDATE: {e}")

update_table_from_dataframe(conn, df_gerentes, update_table, chave, atualizacao)




,IDFUNCIONARIO,NOME,CPF,CARGO,SALARIO,ENDERECO,ID_CIDADE,ID_LOJA,ID_GERENTE,ID_DEPARTAMENTO,SITUACAO,DATA_ADMISSAO,DATA_DEMISSAO,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,59,Isabela Ferreira Santos Bezerra,38962277982,Gerente,3498.54,Rua Santa Elisa - 2458 - Vila Real,4446,121,1,1,Ativa,2013-11-22,None,2013-11-22,2025-03-18,2
1,63,Patricia Barbosa Rodrigues Santos,22413870356,Gerente,3839.87,Rua do Progresso - 4159 - Jardim Independencia,3596,53,2,1,Ativa,2016-11-10,None,2016-11-10,2025-03-18,2
2,98,Carolina Alves Santos Moreira,47930548377,Gerente,3830.56,Rua Santa Julia - 1733 - Jardim Monte Alegre,3221,48,3,1,Ativa,2017-06-18,None,2017-06-18,2025-03-18,2


Tabela 'FUNCIONARIO' atualizada com sucesso!
Erro ao executar: (pyodbc.IntegrityError) ('23000', '[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]A instrução ALTER TABLE conflitou com a restrição do FOREIGN KEY "FK_FUNCIONARIO_GERENTE". O conflito ocorreu no banco de dados "MERCADO_OLTP", tabela "dbo.GERENTE", column \'IDGERENTE\'. (547) (SQLExecDirectW)')
[SQL: ALTER TABLE FUNCIONARIO ADD CONSTRAINT FK_FUNCIONARIO_GERENTE FOREIGN KEY(ID_GERENTE) REFERENCES GERENTE(IDGERENTE)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
